In [1]:
import pandas as pd
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import openpyxl
import matplotlib.pyplot as plt

c:\Users\imoha\AppData\Local\Programs\Python\Python39\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
# Load spreadsheet
xls = pd.ExcelFile('output.xlsx')

# Load a sheet into a DataFrame by its name
df_dict = {sheet_name: xls.parse(sheet_name) for sheet_name in xls.sheet_names}


<hr>

# compine datasets

In [3]:
df_combined = pd.DataFrame()
for name_df, df in df_dict.items():
    df['run_index'], df['manuever'], df['passengers']  = name_df.split('_',2)
    df_combined = pd.concat([df_combined, df], axis=0, ignore_index=True)
    
df_combined

,time,CtrlBus.Chassis.SuspA1.Steer.aSteerWheel,CtrlBus.Chassis.SuspA2.Steer.aSteerWheel,time.1,CtrlBus.Chassis.Damper.L1,CtrlBus.Chassis.Damper.R1,CtrlBus.Chassis.Damper.L2,CtrlBus.Chassis.Damper.R2,CtrlBus.Chassis.Spring.L1,CtrlBus.Chassis.Spring.R1,...,time.46,time.47,time.48,time.49,time.50,time.51,time.52,time.53,time.54,time.55
0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000007,7.973219e-14,-3.985238e-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000021,5.571321e-13,-2.782787e-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000041,3.319326e-12,-1.656251e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000091,3.272846e-11,-1.629042e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50957,34.606476,8.783736e-02,1.174802e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.606476,NaN,34.606476,NaN,34.606476,NaN,34.606476,32.245404,NaN,0.087837
50958,34.721779,8.788645e-02,1.092879e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.721779,NaN,34.721779,NaN,34.721779,NaN,34.721779,31.951595,NaN,0.087886
50959,34.800000,8.791900e-02,1.037667e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.800000,NaN,34.800000,NaN,34.800000,NaN,34.800000,31.754017,NaN,0.087919
50960,34.905042,8.795597e-02,9.639128e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.905042,NaN,34.905042,NaN,34.905042,NaN,34.905042,31.490501,NaN,0.087956


In [4]:
columns_to_keep = ['time','DrvBus.aSteerWheel', 'DrvBus.rAccelPedal', 'DrvBus.rBrakePedal', 'VehBus.Chassis.Body.CG.aRoll', 'VehBus.Chassis.Body.CG.nRoll','run_index', 'manuever', 'passengers']
df_combined = df_combined[columns_to_keep]
df_combined

,time,DrvBus.aSteerWheel,DrvBus.rAccelPedal,DrvBus.rBrakePedal,VehBus.Chassis.Body.CG.aRoll,VehBus.Chassis.Body.CG.nRoll,run_index,manuever,passengers
0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-1.382141e-17,-1.382141e-15,0,DoubleLaneChange,FL
1,0.000007,7.973219e-14,0.000000e+00,2.106936e-09,-2.364362e-10,-2.362727e-08,0,DoubleLaneChange,FL
2,0.000021,5.571321e-13,0.000000e+00,1.052880e-08,-1.180138e-09,-1.178995e-07,0,DoubleLaneChange,FL
3,0.000041,3.319326e-12,0.000000e+00,3.868289e-08,-4.324737e-09,-4.317921e-07,0,DoubleLaneChange,FL
4,0.000091,3.272846e-11,0.000000e+00,1.819939e-07,-2.019735e-08,-2.012997e-06,0,DoubleLaneChange,FL
...,...,...,...,...,...,...,...,...,...
50957,34.606476,8.783736e-02,1.189895e-13,5.799838e-03,-1.081195e-03,-5.547619e-05,55,DoubleLaneChangeISO3888,FL_FR_RL_RR
50958,34.721779,8.788645e-02,0.000000e+00,4.834174e-03,-1.082572e-03,1.872805e-05,55,DoubleLaneChangeISO3888,FL_FR_RL_RR
50959,34.800000,8.791900e-02,3.523133e-15,4.188140e-03,-1.078225e-03,7.740791e-05,55,DoubleLaneChangeISO3888,FL_FR_RL_RR
50960,34.905042,8.795597e-02,0.000000e+00,3.310026e-03,-1.073022e-03,3.057484e-05,55,DoubleLaneChangeISO3888,FL_FR_RL_RR


In [5]:
df = df_combined[df_combined['manuever'] == 'DoubleLaneChange']
train_set = df[df['passengers'] != 'FL_FR_RL_RR']
test_set = df[df['passengers'] == 'FL_FR_RL_RR']

<hr>

In [6]:
X_train = train_set[['time', 'DrvBus.aSteerWheel', 'DrvBus.rAccelPedal', 'DrvBus.rBrakePedal']]
y_train = train_set[['VehBus.Chassis.Body.CG.aRoll']]
X_test = test_set[['time', 'DrvBus.aSteerWheel', 'DrvBus.rAccelPedal', 'DrvBus.rBrakePedal']]
y_test = test_set[['VehBus.Chassis.Body.CG.aRoll']]


<hr>

# TPOT

In [7]:
pipeline_optimizer = TPOTRegressor()

In [8]:
pipeline_optimizer = TPOTRegressor(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)

In [9]:
pipeline_optimizer.fit(X_train, y_train)

c:\Users\imoha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                                                                             
Generation 1 - Current best internal CV score: -1.4188068470232318e-06
                                                                             
Generation 2 - Current best internal CV score: -1.4188068470232318e-06
                                                                             
Generation 3 - Current best internal CV score: -1.409454799562888e-06
                                                                              
Generation 4 - Current best internal CV score: -1.3883033392165015e-06
                                                                              
Generation 5 - Current best internal CV score: -1.3883033392165015e-06
                                                                              
Best pipeline: RandomForestRegressor(MinMaxScaler(input_matrix), bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=11, min_samples_split=6, n_estimators=100)


TPOTRegressor(generations=5, population_size=20, random_state=42, verbosity=2)

In [10]:
print(pipeline_optimizer.score(X_test, y_test))

-1.716400854896184e-06


c:\Users\imoha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# pipeline_optimizer.export('tpot_exported_pipeline.py')